In [1]:
import numpy as np
import tensorflow as tf

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('data/titanic_train.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
#Drop Age and Cabin because containing lots of NULLs
df.drop(['Age', 'Cabin'], axis = 1, inplace = True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Embarked     889 non-null    object 
dtypes: float64(1), int64(5), object(4)
memory usage: 69.7+ KB


In [7]:
#Fill 2 na values for Embarked
df.loc[df['Embarked'].isna(), 'Embarked'] = np.random.choice(df['Embarked'], 2)

In [8]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,0,0,373450,8.0500,S


In [9]:
#Drop additional hard to use columns
df.drop(['PassengerId', 'Name', 'Ticket'], axis = 1, inplace = True)

In [10]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop = 'first')
encoded = ohe.fit_transform(df[['Sex', 'Embarked']])

In [11]:
encoded = pd.DataFrame(encoded.toarray(), columns = ohe.get_feature_names())

In [12]:
df = pd.concat([df, encoded], axis = 1).drop(['Sex', 'Embarked'], axis = 1)

In [13]:
df.head()

,Survived,Pclass,SibSp,Parch,Fare,x0_male,x1_Q,x1_S
0,0,3,1,0,7.2500,1.0,0.0,1.0
1,1,1,1,0,71.2833,0.0,0.0,0.0
2,1,3,0,0,7.9250,0.0,0.0,1.0
3,1,1,1,0,53.1000,0.0,0.0,1.0
4,0,3,0,0,8.0500,1.0,0.0,1.0


In [14]:
def Standardize(df):
    return (df - df.mean())/df.std()

In [15]:
df['Fare'] = df[['Fare']].apply(Standardize)

In [16]:
df_input = df.drop('Survived', axis = 1)
df_output = df['Survived']

In [17]:
ds = tf.data.Dataset.from_tensor_slices((df_input, df_output))

In [18]:
ds.element_spec

(TensorSpec(shape=(7,), dtype=tf.float64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [19]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(21, activation = 'sigmoid', input_shape = (7,)))
model.add(tf.keras.layers.Dense(21, activation = 'sigmoid', kernel_regularizer = tf.keras.regularizers.l1()))
model.add(tf.keras.layers.Dense(1))

model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits = True), metrics = ['accuracy'])

In [20]:
ds_batch = ds.shuffle(1000).batch(5)

In [21]:
model.fit(ds_batch, epochs = 20)

Epoch 1/20
179/179 [==============================] - 0s 1ms/step - loss: 1.2175 - accuracy: 0.6162
Epoch 2/20
179/179 [==============================] - 0s 1ms/step - loss: 0.8244 - accuracy: 0.6162
Epoch 3/20
179/179 [==============================] - 0s 1ms/step - loss: 0.6928 - accuracy: 0.6162
Epoch 4/20
179/179 [==============================] - 0s 1ms/step - loss: 0.6734 - accuracy: 0.6162
Epoch 5/20
179/179 [==============================] - 0s 1ms/step - loss: 0.6700 - accuracy: 0.6162
Epoch 6/20
179/179 [==============================] - 0s 1ms/step - loss: 0.6697 - accuracy: 0.6162
Epoch 7/20
179/179 [==============================] - 0s 1ms/step - loss: 0.6696 - accuracy: 0.6162
Epoch 8/20
179/179 [==============================] - 0s 1ms/step - loss: 0.6702 - accuracy: 0.6162
Epoch 9/20
179/179 [==============================] - 0s 1ms/step - loss: 0.6697 - accuracy: 0.6162
Epoch 10/20
179/179 [==============================] - 0s 1ms/step - loss: 0.6699 - accuracy: 0.6162

In [22]:
model.evaluate(ds_batch)

179/179 [==============================] - 0s 942us/step - loss: 0.6699 - accuracy: 0.6162


[0.6699334979057312, 0.6161616444587708]